In [1]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

from statistics import mean 

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

from xgboost import XGBClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
# Importing the required packages 

In [3]:
df1 = pd.read_csv('training_dataset.csv')
df2 = pd.read_csv('validation_dataset.csv')

In [4]:
final_test_dataset = pd.read_csv('test_dataset.csv')
# This is the final test data. Will be used only for testing at the end

In [5]:
frames = [df1, df2] 
final_training_dataset = pd.concat(frames)
# Merging the training and the validation set into 1 dataset -final_training_dataset

In [6]:
# final_training_dataset

In [7]:
from sklearn import preprocessing
final_training_dataset['FIRE_DISCOVERY_MONTH'] = pd.DatetimeIndex(final_training_dataset['DISCOVERY_DATE']).month
final_training_dataset['DAY_OF_WEEK'] = pd.to_datetime(final_training_dataset['DISCOVERY_DATE']).dt.day_name()

In [8]:
final_training_dataset.columns

Index(['FOD_ID', 'NWCG_REPORTING_AGENCY', 'SOURCE_REPORTING_UNIT_NAME',
       'FIRE_YEAR', 'DISCOVERY_DATE', 'STAT_CAUSE_DESCR', 'FIRE_SIZE',
       'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_DESCR', 'STATE',
       'DISCOVERY_TIME', 'CONT_DATE', 'CONT_TIME', 'COUNTY', 'STAT_CAUSE_CODE',
       'FIRE_DISCOVERY_MONTH', 'DAY_OF_WEEK'],
      dtype='object')

In [9]:
final_training_dataset = final_training_dataset[['FOD_ID', 'NWCG_REPORTING_AGENCY', 'SOURCE_REPORTING_UNIT_NAME',
       'FIRE_YEAR', 'FIRE_DISCOVERY_MONTH',"DAY_OF_WEEK",'DISCOVERY_DATE', 'FIRE_SIZE',
       'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_DESCR', 'STATE',
       'DISCOVERY_TIME', 'CONT_DATE', 'CONT_TIME', 'COUNTY','STAT_CAUSE_DESCR', 
       'STAT_CAUSE_CODE']]

# changing the order of the columns 

In [10]:

from sklearn.preprocessing import LabelEncoder

In [11]:

enc=LabelEncoder()

In [12]:
final_training_dataset['DAY_OF_WEEK'] = enc.fit_transform(final_training_dataset['DAY_OF_WEEK'])
# final_training_dataset['MONTH'] = enc.fit_transform(final_training_dataset['MONTH'])
final_training_dataset['STATE'] = enc.fit_transform(final_training_dataset['STATE'])
final_training_dataset['FIRE_SIZE_CLASS'] = enc.fit_transform(final_training_dataset['FIRE_SIZE_CLASS'])
final_training_dataset['NWCG_REPORTING_AGENCY'] = enc.fit_transform(final_training_dataset['NWCG_REPORTING_AGENCY'])
final_training_dataset['SOURCE_REPORTING_UNIT_NAME'] = enc.fit_transform(final_training_dataset['SOURCE_REPORTING_UNIT_NAME'])

In [13]:
X = final_training_dataset[['FIRE_YEAR','FIRE_DISCOVERY_MONTH','FIRE_SIZE','SOURCE_REPORTING_UNIT_NAME','NWCG_REPORTING_AGENCY',
                           'DAY_OF_WEEK','STATE', "FIRE_SIZE_CLASS"]] # Features
y = final_training_dataset[["STAT_CAUSE_CODE"]] # Target variable

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=10)

In [15]:
clf_rf = RandomForestClassifier()
clf_rf = clf_rf.fit(X_train, y_train)
        
y_pred = clf_rf.predict(X_test)
print(f1_score(y_test, y_pred, average="micro"))

C:\Users\ninadgorey\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.4988201276256315


In [16]:
[estimator.tree_.max_depth for estimator in clf_rf.estimators_]

[40,
 42,
 39,
 39,
 39,
 43,
 38,
 42,
 40,
 40,
 39,
 42,
 43,
 41,
 39,
 41,
 39,
 41,
 43,
 41,
 41,
 42,
 41,
 40,
 38,
 40,
 42,
 38,
 40,
 40,
 40,
 42,
 41,
 39,
 39,
 39,
 40,
 42,
 38,
 39,
 36,
 38,
 38,
 39,
 38,
 41,
 38,
 41,
 40,
 39,
 39,
 39,
 37,
 40,
 41,
 38,
 39,
 41,
 39,
 36,
 40,
 45,
 44,
 40,
 42,
 38,
 41,
 41,
 40,
 40,
 39,
 41,
 36,
 38,
 39,
 40,
 39,
 43,
 42,
 41,
 40,
 44,
 42,
 40,
 39,
 40,
 41,
 42,
 40,
 41,
 37,
 41,
 42,
 40,
 42,
 39,
 43,
 42,
 39,
 38]

### Hypertuning the parameters

In [17]:
rfc = RandomForestClassifier(random_state=10)
params = {'bootstrap': ['True', 'False'], 'max_depth': [35,37,39,41,43], 'n_estimators': [10, 50, 100, 200, 500], 'max_features': [1,2,3,4,5,6,7,8]}
clf = RandomizedSearchCV(rfc, params, n_jobs=-1, verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 24.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 28.9min finished
C:\Users\ninadgorey\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
            

In [18]:
print(clf.best_params_)
print(clf.best_score_)
print(clf.best_estimator_)

{'n_estimators': 500, 'max_features': 6, 'max_depth': 41, 'bootstrap': 'False'}
0.5003365152946587
RandomForestClassifier(bootstrap='False', ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=41, max_features=6,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)


In [19]:
clf_rf = RandomForestClassifier(n_estimators= 500, max_features= 3, max_depth= 37,bootstrap= "False" )
clf_rf = clf_rf.fit(X_train, y_train.values.ravel())
        

In [20]:
final_test_dataset.columns

Index(['FOD_ID', 'NWCG_REPORTING_AGENCY', 'SOURCE_REPORTING_UNIT_NAME',
       'FIRE_YEAR', 'DISCOVERY_DATE', 'STAT_CAUSE_DESCR', 'FIRE_SIZE',
       'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_DESCR', 'STATE',
       'DISCOVERY_TIME', 'CONT_DATE', 'CONT_TIME', 'COUNTY',
       'STAT_CAUSE_CODE'],
      dtype='object')

In [21]:
from sklearn import preprocessing
final_test_dataset['FIRE_DISCOVERY_MONTH'] = pd.DatetimeIndex(final_test_dataset['DISCOVERY_DATE']).month
final_test_dataset['DAY_OF_WEEK'] = pd.to_datetime(final_test_dataset['DISCOVERY_DATE']).dt.day_name()

In [22]:
final_test_dataset['DAY_OF_WEEK'] = enc.fit_transform(final_test_dataset['DAY_OF_WEEK'])
final_test_dataset['STATE'] = enc.fit_transform(final_test_dataset['STATE'])
final_test_dataset['FIRE_SIZE_CLASS'] = enc.fit_transform(final_test_dataset['FIRE_SIZE_CLASS'])
final_test_dataset['NWCG_REPORTING_AGENCY'] = enc.fit_transform(final_test_dataset['NWCG_REPORTING_AGENCY'])
final_test_dataset['SOURCE_REPORTING_UNIT_NAME'] = enc.fit_transform(final_test_dataset['SOURCE_REPORTING_UNIT_NAME'])

In [23]:
X = final_test_dataset[['FIRE_YEAR','FIRE_DISCOVERY_MONTH','FIRE_SIZE','SOURCE_REPORTING_UNIT_NAME','NWCG_REPORTING_AGENCY',
                           'DAY_OF_WEEK','STATE', "FIRE_SIZE_CLASS"]] # Features
y = final_test_dataset[["STAT_CAUSE_CODE"]] # Target variable

In [24]:
y_pred = clf_rf.predict(X)

In [25]:
# print("Accuracy score for Random Forest  classifier is:",accuracy_score(y,y_pred))
print(f1_score(y, y_pred, average="micro"))

0.3062309727336781


### KNN:


In [26]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train.values.ravel())
# knn_prediction = knn_model.predict(oob_X)
    

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [27]:
knn_prediction = knn_model.predict(X_test)

In [28]:
print(f1_score(y_test, knn_prediction, average="micro"))

0.48567535229992026


In [29]:
knn_prediction = knn_model.predict(X)

In [30]:
print(f1_score(y, knn_prediction, average="micro"))

0.16475452333789767


#### Experiments with PCA

In [32]:
from sklearn.decomposition import PCA

In [33]:
X = final_training_dataset[['FIRE_YEAR','FIRE_DISCOVERY_MONTH','FIRE_SIZE','SOURCE_REPORTING_UNIT_NAME','NWCG_REPORTING_AGENCY',
                           'DAY_OF_WEEK','STATE', "FIRE_SIZE_CLASS"]] # Features
y = final_training_dataset[["STAT_CAUSE_CODE"]] # Target variable

In [36]:
pca = PCA(n_components = 0.95)
df3SS_PCA = pca.fit_transform(X)
# Applying PCA to the standardized data (df3SS)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df3SS_PCA,y, test_size=.2, random_state=10)

In [39]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train.values.ravel())
# knn_prediction = knn_model.predict(oob_X)
    

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [40]:
knn_prediction = knn_model.predict(X_test)

In [41]:
print(f1_score(y_test, knn_prediction, average="micro"))

0.4686752193565541


In [43]:
X = final_test_dataset[['FIRE_YEAR','FIRE_DISCOVERY_MONTH','FIRE_SIZE','SOURCE_REPORTING_UNIT_NAME','NWCG_REPORTING_AGENCY',
                           'DAY_OF_WEEK','STATE', "FIRE_SIZE_CLASS"]] # Features
y = final_test_dataset[["STAT_CAUSE_CODE"]] # Target variable

In [44]:
pca = PCA(n_components = 0.95)
df3SS_PCA = pca.fit_transform(X)
# Applying PCA to the standardized data (df3SS)

In [45]:
knn_prediction = knn_model.predict(df3SS_PCA)

In [46]:
print(f1_score(y, knn_prediction, average="micro"))

0.16771912099014902
